# Lab 3 — Statistical Language Models (Practical Dev Pipeline)

This notebook teaches **how a developer would actually train, evaluate,
and use a Statistical Language Model (SLM)** using a real dataset.

The goal is not just theory, but a **repeatable NLP pipeline**.

---

## What you will learn

- Loading and preparing a real corpus
- Train / validation / test splitting
- Vocabulary construction and OOV handling
- Training unigram, bigram, and trigram models
- Smoothing and why it is required
- Evaluating with perplexity
- Using an SLM for next-word prediction


## 1. Setup


In [3]:

import nltk
import re
import math
import random
from collections import Counter
from typing import List

nltk.download("reuters")


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

https://trec.nist.gov/data/reuters/reuters.html

## 2. Load a real dataset (Reuters)


In [4]:

from nltk.corpus import reuters

# reuters.fileids() returns IDs for all documents.
# reuters.raw(fid) returns raw text for each document.
# We collect all documents into a Python list.

docs = [reuters.raw(fid) for fid in reuters.fileids()]
len(docs)


10788

## 3. Train / validation / test split


In [5]:

random.seed(42)  # Fixes randomness for reproducibility.
random.shuffle(docs) #  Prevents topic or time-based bias in splits.

train_docs = docs[:6000]  # Training set: estimate probabilities
val_docs   = docs[6000:7000]  # Validation set: tune hyperparameters (smoothing, vocab size)
test_docs  = docs[7000:8000]  # Test set: final, unbiased evaluation

len(train_docs), len(val_docs), len(test_docs)


(6000, 1000, 1000)

## 4. Tokenization and normalization


In [6]:
print(train_docs [0])

LOWER U.S. SOYBEAN LOAN IDEA SHARPLY CRITICIZED
  U.S. soybean lobbyists and
  congressional aides criticized a proposal from a senior
  Agriculture Department official that Congress allow the U.S.
  soybean loan level to be officially lowered to 4.56 dlrs per
  bushel next year.
      "I don't know who in Congress would propose that happening.
  Politically it would be totally unacceptable," an aide to a
  senior farm-state senator said.
      USDA undersecretary Daniel Amstutz said this week that
  Congress should give USDA authority to keep the soybean loan 
  at its current effective rate of 4.56 dlrs per bushel rather
  than increasing it to its minimum allowed level of 4.77 dlrs.
      "I'm convinced that Congress will not go along with this,"
  American Soybean Association President Dave Haggard said.
      Amstutz told reporters following a senate hearing that if
  the soybean loan rate were 4.56 dlrs, USDA could then consider
  ways to make U.S. soybeans more competitive.
    

In [7]:
# Defines a reusable tokenization function.
# Explicit, deterministic preprocessing is critical in NLP pipelines.

def tokenize(text: str) -> List[str]:
    text = text.lower()  # Lowercases text to reduce vocabulary size.
    text = re.sub(r"[^a-z0-9\s]+", " ", text)  # Removes punctuation. Keeps letters and numbers. Numbers matter in Reuters (prices, years, quantities).
    text = re.sub(r"\s+", " ", text).strip()  # Collapses multiple spaces. Removes leading and trailing whitespace.
    return text.split()  # Converts string into a list of tokens. Splits on spaces.

tokenize(train_docs[0])[:20]


['lower',
 'u',
 's',
 'soybean',
 'loan',
 'idea',
 'sharply',
 'criticized',
 'u',
 's',
 'soybean',
 'lobbyists',
 'and',
 'congressional',
 'aides',
 'criticized',
 'a',
 'proposal',
 'from',
 'a']

## 5. Vocabulary and OOV handling


In [8]:

train_tokens = []
for d in train_docs:
    train_tokens.extend(tokenize(d))  # Tokenizes every training document. Flattens them into a single list.

freq = Counter(train_tokens)  # Counts word frequencies in the training set. This frequency distribution defines the vocabulary.
print(freq.most_common(10))  # Displays the 10 most common words and their counts

MIN_COUNT = 5  # Threshold for keeping words. Rare words cause unreliable probability estimates.
vocab = {w for w, c in freq.items() if c >= MIN_COUNT}  # Keeps only words that occur at least MIN_COUNT times. Keeps only sufficiently frequent tokens. Reduces noise and model size.
""" Why these tokens are mandatory
<UNK>: unknown words at inference time
<s>: sentence start
</s>: sentence end 
Without them:
SLMs break on unseen text
Sentence modeling becomes impossible"""
vocab |= {"<UNK>", "<s>", "</s>"}

len(vocab)


[('the', 38163), ('of', 20464), ('to', 19921), ('in', 16238), ('and', 14100), ('said', 14089), ('a', 13811), ('mln', 10604), ('s', 8564), ('vs', 8149)]


8773

In [9]:
train_tokens

['lower',
 'u',
 's',
 'soybean',
 'loan',
 'idea',
 'sharply',
 'criticized',
 'u',
 's',
 'soybean',
 'lobbyists',
 'and',
 'congressional',
 'aides',
 'criticized',
 'a',
 'proposal',
 'from',
 'a',
 'senior',
 'agriculture',
 'department',
 'official',
 'that',
 'congress',
 'allow',
 'the',
 'u',
 's',
 'soybean',
 'loan',
 'level',
 'to',
 'be',
 'officially',
 'lowered',
 'to',
 '4',
 '56',
 'dlrs',
 'per',
 'bushel',
 'next',
 'year',
 'i',
 'don',
 't',
 'know',
 'who',
 'in',
 'congress',
 'would',
 'propose',
 'that',
 'happening',
 'politically',
 'it',
 'would',
 'be',
 'totally',
 'unacceptable',
 'an',
 'aide',
 'to',
 'a',
 'senior',
 'farm',
 'state',
 'senator',
 'said',
 'usda',
 'undersecretary',
 'daniel',
 'amstutz',
 'said',
 'this',
 'week',
 'that',
 'congress',
 'should',
 'give',
 'usda',
 'authority',
 'to',
 'keep',
 'the',
 'soybean',
 'loan',
 'at',
 'its',
 'current',
 'effective',
 'rate',
 'of',
 '4',
 '56',
 'dlrs',
 'per',
 'bushel',
 'rather',
 'tha

In [10]:
vocab

{'271',
 'pioneer',
 '188',
 '909',
 '769',
 'entire',
 '736',
 'unc',
 '166',
 'conservation',
 'closes',
 'incurred',
 'planting',
 'na',
 'mineworkers',
 '024',
 'exports',
 'rescue',
 'preussag',
 'shield',
 'escorting',
 'equities',
 'bulolo',
 '17',
 'mobex',
 'foreigners',
 'leases',
 'cease',
 'pay',
 'compounding',
 'cutting',
 'cds',
 'regularly',
 'threatened',
 '835',
 'option',
 'rand',
 'outbreaks',
 'pakistani',
 '30',
 'power',
 'taikoo',
 '719',
 'per',
 'grounded',
 '594',
 'quaker',
 'member',
 '960',
 'afford',
 '600',
 'pursue',
 'practice',
 'sage',
 'schillings',
 '32',
 'exercised',
 'claiming',
 '258',
 'consortium',
 '35',
 'ems',
 '1987',
 '829',
 'rejects',
 '788',
 'preceding',
 'controlled',
 'chapman',
 'rally',
 'turnaround',
 'extract',
 'unwillingness',
 'seeks',
 'jacksons',
 '224',
 'hydro',
 'used',
 'tallow',
 'enterprises',
 'announcement',
 'hutton',
 'ambrit',
 'foods',
 'believes',
 '040',
 'supplying',
 'berisford',
 'cie',
 'features',
 'relo

In [11]:
list(vocab).count("gain")

1

In [12]:
# Count words in vocab
vocab_counts = {w: freq[w] for w in vocab}
vocab_counts

{'271': 20,
 'pioneer': 23,
 '188': 25,
 '909': 10,
 '769': 5,
 'entire': 42,
 '736': 8,
 'unc': 13,
 '166': 27,
 'conservation': 34,
 'closes': 12,
 'incurred': 19,
 'planting': 46,
 'na': 27,
 'mineworkers': 7,
 '024': 6,
 'exports': 672,
 'rescue': 7,
 'preussag': 13,
 'shield': 7,
 'escorting': 8,
 'equities': 14,
 'bulolo': 5,
 '17': 560,
 'mobex': 7,
 'foreigners': 9,
 'leases': 14,
 'cease': 7,
 'pay': 526,
 'compounding': 5,
 'cutting': 67,
 'cds': 7,
 'regularly': 7,
 'threatened': 46,
 '835': 10,
 'option': 113,
 'rand': 26,
 'outbreaks': 5,
 'pakistani': 6,
 '30': 885,
 'power': 122,
 'taikoo': 5,
 '719': 15,
 'per': 1172,
 'grounded': 6,
 '594': 11,
 'quaker': 23,
 'member': 149,
 '960': 22,
 'afford': 10,
 '600': 194,
 'pursue': 29,
 'practice': 17,
 'sage': 6,
 'schillings': 8,
 '32': 240,
 'exercised': 14,
 'claiming': 6,
 '258': 16,
 'consortium': 25,
 '35': 360,
 'ems': 35,
 '1987': 1389,
 '829': 11,
 'rejects': 35,
 '788': 17,
 'preceding': 8,
 'controlled': 61,
 'cha

## 6. Sentence boundaries


In [13]:

def add_boundaries(tokens: List[str], n: int):
    return ["<s>"]*(n-1) + tokens + ["</s>"]


## 7. Replace OOV tokens


In [14]:

def replace_oov(tokens: List[str]):

    """Centralizes unknown-word handling. Ensures consistency across train/val/test.
    Any unseen word becomes <UNK>. Prevents KeyErrors and zero-probability issues."""
    return [t if t in vocab else "<UNK>" for t in tokens]


## 8. Train n-gram models


In [15]:

def get_ngrams(tokens: List[str], n: int):
    """Generates n-grams from a list of tokens."""
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]  # Sliding window over tokens to create n-grams.

def train_ngram(docs, n):
    """Trains an n-gram language model by counting n-grams and their contexts."""
    ngram_counts = Counter()  # Counts of full n-grams
    context_counts = Counter()  # Counts of (n-1)-gram contexts

    # Both are needed to compute conditional probabilities.

    for d in docs:
        toks = replace_oov(tokenize(d))  # Tokenizes and replaces out-of-vocab words with <UNK>.
        toks = add_boundaries(toks, n)  # Adds sentence boundary tokens.
        for ng in get_ngrams(toks, n):  # Generates n-grams from the token list.
            ngram_counts[ng] += 1  # Increments the count for the full n-gram.
            context_counts[ng[:-1]] += 1  # Increments the count for the (n-1)-gram context.

    return ngram_counts, context_counts

uni_counts, uni_ctx = train_ngram(train_docs, 1)  # Trains unigram model.
bi_counts, bi_ctx   = train_ngram(train_docs, 2)  # Trains bigram model.
tri_counts, tri_ctx = train_ngram(train_docs, 3)  # Trains trigram model.


In [16]:
bi_counts

Counter({('in', 'the'): 3712,
         ('of', 'the'): 3711,
         ('lt', '<UNK>'): 3205,
         ('u', 's'): 3056,
         ('said', 'the'): 2979,
         ('mln', 'dlrs'): 2533,
         ('said', 'it'): 2457,
         ('mln', 'vs'): 2246,
         ('<UNK>', '<UNK>'): 1919,
         ('cts', 'vs'): 1839,
         ('the', 'company'): 1788,
         ('for', 'the'): 1551,
         ('000', 'vs'): 1413,
         ('he', 'said'): 1384,
         ('to', 'the'): 1344,
         ('the', '<UNK>'): 1275,
         ('cts', 'net'): 1228,
         ('the', 'u'): 1190,
         ('on', 'the'): 1084,
         ('<s>', '<UNK>'): 1005,
         ('vs', 'loss'): 1003,
         ('<UNK>', 'and'): 993,
         ('dlrs', 'in'): 951,
         ('pct', 'of'): 942,
         ('billion', 'dlrs'): 913,
         ('to', 'be'): 911,
         ('<UNK>', 'said'): 897,
         ('it', 'said'): 882,
         ('and', 'the'): 878,
         ('<UNK>', 'the'): 872,
         ('and', '<UNK>'): 872,
         ('000', 'dlrs'): 869,
     

#### Using NLTK

In [17]:
from nltk.util import ngrams
from collections import Counter

def get_ngrams_nltk(tokens: List[str], n: int):
    return list(ngrams(tokens, n))

def train_ngram_nltk(docs, n):
    """Trains an n-gram language model using NLTK's ngrams."""
    
    ngram_counts = Counter()
    context_counts = Counter()

    for d in docs:
        # Step 1: tokenize
        toks = tokenize(d)
        
        # Step 2: replace OOV
        toks = replace_oov(toks)
        
        # Step 3: add sentence boundaries
        toks = add_boundaries(toks, n)

        # Step 4: generate n-grams using NLTK
        for ng in ngrams(toks, n):
            ngram_counts[ng] += 1
            context_counts[ng[:-1]] += 1

    return ngram_counts, context_counts


In [18]:
uni_counts, uni_ctx = train_ngram_nltk(train_docs, 1)
bi_counts, bi_ctx   = train_ngram_nltk(train_docs, 2)
tri_counts, tri_ctx = train_ngram_nltk(train_docs, 3)


## 9. Add-k smoothing


Add-k smoothing is a simple technique used in statistical language models to avoid zero probabilities for unseen events.
In an n-gram language model, probabilities are estimated from counts.

For a bigram model:

P(w∣h)= count(h, w) / count(h)
	​


If a word sequence never appeared in the training data, then:

- count(h, w) = 0

- so P(w | h) = 0

This causes two serious problems:

Zero probability chains
If any word in a sentence has probability 0, the probability of the entire sentence becomes 0.

Infinite perplexity
Log probability of 0 is undefined, so evaluation breaks.

In [19]:

def prob_addk(ngram, ngram_counts, context_counts, k=0.5):
    """ Computes add-k smoothed probability for an n-gram."""
    V = len(vocab)  # Vocabulary size for smoothing.
    return (ngram_counts[ngram] + k) / (context_counts[ngram[:-1]] + k*V)  # Add-k smoothing formula.


## 10. Perplexity evaluation


In [20]:
# Evaluate perplexity on test set
def perplexity(docs, n, ngram_counts, context_counts, k=0.5):
    """ Calculates perplexity of the given n-gram model on the provided documents."""
    log_prob = 0  # Total log probability of the test set.
    count = 0  # Total number of n-grams in the test set.

    for d in docs:
        toks = replace_oov(tokenize(d))
        toks = add_boundaries(toks, n)
        for ng in get_ngrams(toks, n):
            p = prob_addk(ng, ngram_counts, context_counts, k)  # Probability of the n-gram.
            log_prob += math.log2(p)  # Use log2 for perplexity calculation to avoid large numbers.
            count += 1

    H = -log_prob / count  # Compute cross-entropy
    return 2**H  # Perplexity is 2 raised to the cross-entropy.

# Compare perplexities. Lower perplexity is better."

print("Unigram:", perplexity(test_docs, 1, uni_counts, uni_ctx))
print("Bigram :", perplexity(test_docs, 2, bi_counts, bi_ctx))
print("Trigram:", perplexity(test_docs, 3, tri_counts, tri_ctx))


Unigram: 770.7568679909757
Bigram : 532.3717185195632
Trigram: 2556.0951062679683


## 11. Next-word prediction


In [21]:

def next_word(context, n, ngram_counts, context_counts, top_k=5):
    """ Predicts the top_k next words given the context using the n-gram model."""
    context = tuple(context[-(n-1):]) if n > 1 else tuple()  # Uses last (n-1) words as context.
    scores = []
    for w in vocab:
        if w == "<s>": # Prevents predicting start-of-sentence token
            continue
        ng = context + (w,)  # Forms the candidate n-gram.
        scores.append((w, prob_addk(ng, ngram_counts, context_counts)))  # Scores the candidate word.
    scores.sort(key=lambda x: -x[1])
    return scores[:top_k]

next_word(["oil", "prices"], 3, tri_counts, tri_ctx)


[('and', 0.004929346040092014),
 ('to', 0.001643115346697338),
 ('would', 0.001643115346697338),
 ('was', 0.0014240333004710264),
 ('are', 0.0012049512542447146)]

In [22]:
print("oil in vocab:", "oil" in vocab)
print("prices in vocab:", "prices" in vocab)


oil in vocab: True
prices in vocab: True


In [23]:
print("Context count tri_ctx[('oil','prices')]:", tri_ctx[("oil","prices")])


Context count tri_ctx[('oil','prices')]: 178


In [24]:
candidates = []
for w in vocab:
    if w in {"<s>"}:
        continue
    candidates.append((w, tri_counts[("oil","prices",w)]))

sorted(candidates, key=lambda x: -x[1])[:20]


[('and', 22),
 ('to', 7),
 ('would', 7),
 ('was', 6),
 ('are', 5),
 ('in', 5),
 ('</s>', 5),
 ('rose', 5),
 ('but', 5),
 ('were', 5),
 ('the', 5),
 ('should', 4),
 ('from', 4),
 ('which', 4),
 ('by', 4),
 ('on', 4),
 ('<UNK>', 3),
 ('last', 3),
 ('it', 3),
 ('at', 3)]

## Why this pipeline matters

This notebook mirrors real NLP systems:

- deterministic preprocessing
- explicit vocabulary
- train/validate/test separation
- quantitative evaluation
- API-style inference

Neural language models build on the same principles.
